Based on:

https://www.dlology.com/blog/how-to-train-an-object-detection-model-easy-for-free/

https://towardsdatascience.com/deeppicar-part-6-963334b2abe0


Pretrained Model from (Tensorflow detection model zoo):

https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md

# Section 0: Actions done before running this notebook
1- Took around 300 photos of cars with lables

2- Run script to scale images dimentions to 800x600 and Divide images to 80% train, 10% test, 10% validation

3- Use [labelImg](https://github.com/tzutalin/labelImg) for annotating "plate no" and "bus run" objects on each of train & test images

In [1]:
# These versions required to work with the scripts available in https://github.com/tensorflow/models/tree/master/research/object_detection
# for transfer learning
!pip install numpy==1.17.4
%tensorflow_version 1.x
!pip install --user gast==0.2.2

TensorFlow 1.x selected.


# Section 1: Mount Google drive
Mount my Google Drive to save modeling output files there, so that it won't be wiped out when colab Virtual Machine restarts.

In [2]:
import os
from google.colab import drive
drive.mount('/content/gdrive')
model_dir = '/content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource'
#!rm -rf '{model_dir}'
os.makedirs(model_dir, exist_ok=True)
!ls -ltra '{model_dir}'/..

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
total 72
drwx------ 2 root root  4096 Mar  5 23:36  Papers
drwx------ 2 root root  4096 Mar  6 03:31  LevelCrossingRemovalProject
drwx------ 2 root root  4096 Mar 25 04:14  Trial
-rw------- 1 root root  3414 Apr 15 08:08  ghawadykey
-rw------- 1 root root   762 Apr 21 06:50  ghawadykey.pub
drwx------ 2 root root  4096 Apr 22 07:51  SampleClicks
-rw------- 1 root root 20284 Apr 22 07:53 'Bus Run Label.docx'
drwx------ 2 root root  4096 Apr 22 07:55  FinalSourc

# Section 2: Configs and Hyperparameters

Support a variety of models, you can find more pretrained model from [Tensorflow detection model zoo: COCO-trained models](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md#coco-trained-models), as well as their pipline config files in [object_detection/samples/configs/](https://github.com/tensorflow/models/tree/master/research/object_detection/samples/configs).

In [0]:
# Number of training steps
num_steps = 5000  # 200000

# Number of evaluation steps
num_eval_steps = 50


# model name and configs are from Model Zoo github: 
# https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md#coco-trained-models
# https://github.com/tensorflow/models/tree/master/research/object_detection/samples/configs
MODELS_CONFIG = {
    'ssdlite_mobilenet_v2': {
        'model_name': 'ssdlite_mobilenet_v2_coco_2018_05_09',
        'pipeline_file': 'ssdlite_mobilenet_v2_coco.config',
        'batch_size': 12
    },
    'ssd_mobilenet_v2_quantized': {
        'model_name': 'ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03',
        'pipeline_file': 'ssd_mobilenet_v2_quantized_300x300_coco.config',
        'batch_size': 12
    },
    #http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v3_small_coco_2020_01_14.tar.gz
    'ssd_mobilenet_v3_small_coco': {
        'model_name': 'ssd_mobilenet_v3_small_coco_2020_01_14',
        'pipeline_file': 'ssdlite_mobilenet_v3_small_320x320_coco.config',
        'batch_size': 12
    },
    #https://github.com/tensorflow/models/blob/master/research/object_detection/samples/configs/ssdlite_mobilenet_v3_large_320x320_coco.config
    #http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v3_large_coco_2020_01_14.tar.gz
    'ssd_mobilenet_v3_large_coco': {
        'model_name': 'ssd_mobilenet_v3_large_coco_2020_01_14',
        'pipeline_file': 'ssdlite_mobilenet_v3_large_320x320_coco.config',
        'batch_size': 12
    },
    #https://storage.cloud.google.com/mobilenet_edgetpu/checkpoints/ssdlite_mobilenet_edgetpu_coco_quant.tar.gz
    'ssdlite_mobilenet_edgetpu_coco_quant': {
        'model_name': 'ssdlite_mobilenet_edgetpu_coco_quant',
        'pipeline_file': 'ssdlite_mobilenet_edgetpu_320x320_coco_quant.config',
        'batch_size': 12
    },
    'faster_rcnn_inception_v2': {
        'model_name': 'faster_rcnn_inception_v2_coco_2018_01_28',
        'pipeline_file': 'faster_rcnn_inception_v2_pets.config',
        'batch_size': 12
    },
    'rfcn_resnet101': {
        'model_name': 'rfcn_resnet101_coco_2018_01_28',
        'pipeline_file': 'rfcn_resnet101_pets.config',
        'batch_size': 12
    }
}

# Select a model in MODELS_CONFIG
# Note: Must be a quantized model, which reduces the model size significantly for mobile/edge use
selected_model = 'ssd_mobilenet_v2_quantized'
#selected_model = 'ssd_mobilenet_v3_large_coco'  # used 15K training steps to get better results
#selected_model = 'ssd_mobilenet_v3_small_coco'  # didn't give good results
#selected_model = 'ssdlite_mobilenet_v2'  # the False Positives for plate numbers were almost in every images

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

# Name of the pipline file in tensorflow object detection API.
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

# Training batch size fits in Colabe's Tesla K80 GPU memory for selected model.
batch_size = MODELS_CONFIG[selected_model]['batch_size']

# Section 3: Set up Training Environment

In [4]:
# Where the new trained model will be saved
from datetime import datetime
curdate = datetime.now().strftime("%Y_%m_%d")

trainedmodel_dir = os.path.join(model_dir, 'trainedmodel', selected_model, curdate)
os.makedirs(trainedmodel_dir, exist_ok = True)

print(trainedmodel_dir)

/content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource/trainedmodel/ssd_mobilenet_v2_quantized/2020_04_30


## Install required packages

In [7]:
%cd '{model_dir}'
#!git clone --quiet https://github.com/tensorflow/models.git
print('installing protobuf')
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk
print('installing Cython')
!pip install -q Cython contextlib2 pillow lxml matplotlib
print('installing pycocotools')
!pip install -q pycocotools
print('run protoc')
%cd '{model_dir}/models/research'
!protoc object_detection/protos/*.proto --python_out=.
print('setting environment variable')
import os
os.environ['PYTHONPATH'] += ':/content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource/models/research/:/content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource/models/research/slim/'

print('run model_builder_test')
# To verify all dependencies are successfull installed
#!python object_detection/builders/model_builder_test.py

/content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource
installing protobuf
installing Cython
installing pycocotools
run protoc
/content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource/models/research
setting environment variable
run model_builder_test


## Prepare `tfrecord` files

Use the following scripts to generate the `tfrecord` files which is used for model training and evaluation.

In [8]:
%cd {model_dir}
print("train_cars xml_to_csv")
# Convert train folder annotation xml files to a single csv file,
# generate the `label_map.pbtxt` file to `data/` directory as well.
!python code/xml_to_csv.py -i Data/train_cars -o Data/annotations/train_labels.csv -l Data/annotations
print("test_cars xml_to_csv")
# Convert test folder annotation xml files to a single csv.
!python code/xml_to_csv.py -i Data/test_cars -o Data/annotations/test_labels.csv

print("train_labels generate_tfrecord")
# Generate `train.record`
!python code/generate_tfrecord.py --csv_input=Data/annotations/train_labels.csv --output_path=Data/annotations/train.record --img_path=Data/train_cars --label_map Data/annotations/label_map.pbtxt
print("test_labels generate_tfrecord")
# Generate `test.record`
!python code/generate_tfrecord.py --csv_input=Data/annotations/test_labels.csv --output_path=Data/annotations/test.record --img_path=Data/test_cars --label_map Data/annotations/label_map.pbtxt

/content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource
train_cars xml_to_csv
Successfully converted xml to csv.
Generate `Data/annotations/label_map.pbtxt`
test_cars xml_to_csv
Successfully converted xml to csv.
train_labels generate_tfrecord


W0429 13:49:09.578909 140207477233536 module_wrapper.py:139] From code/generate_tfrecord.py:107: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.


W0429 13:49:10.714112 140207477233536 module_wrapper.py:139] From /content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource/models/research/object_detection/utils/label_map_util.py:138: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

Successfully created the TFRecords: /content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource/Data/annotations/train.record
test_labels generate_tfrecord


W0429 13:51:12.708905 139934040639360 module_wrapper.py:139] From co

In [0]:
test_record_fname = model_dir + '/Data/annotations/test.record'
train_record_fname = model_dir + '/Data/annotations/train.record'
label_map_pbtxt_fname = model_dir + '/Data/annotations/label_map.pbtxt'

In [10]:
!cat Data/annotations/test_labels.csv

filename,width,height,class,xmin,ymin,xmax,ymax
1.jpg,800,600,BusRun,507,193,593,221
1.jpg,800,600,PlateNo,283,460,468,528
2.jpg,800,600,PlateNo,289,494,457,548
2.jpg,800,600,BusRun,482,228,555,249
3.jpg,800,600,PlateNo,213,430,389,490
3.jpg,800,600,BusRun,426,150,504,173
4.jpg,800,600,PlateNo,281,445,459,542
4.jpg,800,600,BusRun,349,148,418,167
5.jpg,800,600,PlateNo,273,415,412,472
5.jpg,800,600,BusRun,459,164,532,189
6.jpg,800,600,PlateNo,306,381,487,436
6.jpg,800,600,BusRun,518,149,597,175
7.jpg,800,600,PlateNo,279,419,428,461
7.jpg,800,600,BusRun,450,156,522,179
8.jpg,800,600,PlateNo,300,433,459,472
8.jpg,800,600,BusRun,467,214,528,234
9.jpg,800,600,PlateNo,358,401,533,446
9.jpg,800,600,BusRun,285,122,360,143
10.jpg,800,600,PlateNo,248,471,417,529
10.jpg,800,600,BusRun,451,198,530,222
11.jpg,800,600,PlateNo,302,396,463,455
11.jpg,800,600,BusRun,248,178,311,204
12.jpg,800,600,PlateNo,305,467,475,514
12.jpg,800,600,BusRun,533,193,609,219
13.jpg,800,600,PlateNo,223,466,381,519
13.jpg,

## Download base model

In [68]:
%cd '{model_dir}/models/research'

/content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource/models/research


In [69]:
import os
import shutil
import glob
import urllib.request
import tarfile
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = model_dir+'/models/research/pretrained_model'

# commented if already done
#'''
print('MODEL_FILE=', MODEL_FILE)
if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

print('extracting model file')
tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)

os.rename(MODEL, DEST_DIR)
#'''

MODEL_FILE= ssd_mobilenet_v3_large_coco_2020_01_14.tar.gz
extracting model file


In [13]:
!pwd

/content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource/models/research


In [70]:
!echo '{DEST_DIR}'
!ls -alh '{DEST_DIR}'

/content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource/models/research/pretrained_model
total 58M
-rw------- 1 root root   77 Jan 15 22:35 checkpoint
-rw------- 1 root root  13M Jan 15 22:35 frozen_inference_graph.pb
-rw------- 1 root root  25M Jan 15 22:35 model.ckpt.data-00000-of-00001
-rw------- 1 root root  12K Jan 15 22:35 model.ckpt.index
-rw------- 1 root root 7.3M Jan 15 22:36 model.ckpt.meta
-rw------- 1 root root  13M Jan 15 22:36 model.tflite
-rw------- 1 root root 4.7K Jan 15 22:36 pipeline.config


In [71]:
fine_tune_checkpoint = os.path.join(DEST_DIR, "model.ckpt")
fine_tune_checkpoint

'/content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource/models/research/pretrained_model/model.ckpt'

# Section 4: Transfer Learning Training

## Configuring a Training Pipeline

In [72]:
import os
pipeline_fname = os.path.join(model_dir+'/models/research/object_detection/samples/configs/', pipeline_file)
print(pipeline_fname)
assert os.path.isfile(pipeline_fname), '`{}` not exist'.format(pipeline_fname)

/content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource/models/research/object_detection/samples/configs/ssdlite_mobilenet_v3_large_320x320_coco.config


In [0]:
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

In [0]:
# model's .config file is protocol buffer file so can be edited with via google.protobuf
import sys
import tensorflow as tf
from google.protobuf import text_format
from object_detection.protos import pipeline_pb2

pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()

# read pipeline config file
with tf.gfile.GFile(pipeline_fname, "r") as f:
  proto_str = f.read()
  text_format.Merge(proto_str, pipeline_config)

# pipeline_config will have all config parameters that can be overwritten
pipeline_config.train_input_reader.tf_record_input_reader.input_path[0] = train_record_fname
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[0] = test_record_fname

pipeline_config.train_input_reader.label_map_path = label_map_pbtxt_fname
pipeline_config.eval_input_reader[0].label_map_path = label_map_pbtxt_fname

if pipeline_config.train_config.fine_tune_checkpoint:
  pipeline_config.train_config.fine_tune_checkpoint = fine_tune_checkpoint
pipeline_config.train_config.batch_size = batch_size
pipeline_config.train_config.num_steps = num_steps

# Depending on the base model type, currently there is either ssd or faster_rcnn
num_classes = get_num_classes(label_map_pbtxt_fname)
if pipeline_config.model.ssd:
  pipeline_config.model.ssd.num_classes = num_classes
elif pipeline_config.model.faster_rcnn:
  pipeline_config.model.faster_rcnn.num_classes = num_classes

# Save updated config
config_text = text_format.MessageToString(pipeline_config)
with tf.gfile.Open(pipeline_fname, "wb") as f:
    f.write(config_text)


In [75]:
!cat '{label_map_pbtxt_fname}'

item {
    id: 1
    name: 'BusRun'
}

item {
    id: 2
    name: 'PlateNo'
}

In [76]:
# look for num_classes: 6, since we have 5 different road signs and 1 person type (total of 6 types) 
!cat '{pipeline_fname}'

model {
  ssd {
    num_classes: 2
    image_resizer {
      fixed_shape_resizer {
        height: 320
        width: 320
      }
    }
    feature_extractor {
      type: "ssd_mobilenet_v3_large"
      depth_multiplier: 1.0
      min_depth: 16
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 3.9999999e-05
          }
        }
        initializer {
          truncated_normal_initializer {
            mean: 0.0
            stddev: 0.029999999
          }
        }
        activation: RELU_6
        batch_norm {
          decay: 0.97000003
          center: true
          scale: true
          epsilon: 0.001
          train: true
        }
      }
      use_depthwise: true
      override_base_feature_extractor_hyperparams: true
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_thre

## Run Tensorboard

In [77]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

--2020-04-29 15:40:39--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 3.220.228.61, 3.221.30.222, 52.87.72.17, ...
Connecting to bin.equinox.io (bin.equinox.io)|3.220.228.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.7’

ngrok-stable-linux- 100%[===================>]  13.13M  12.7MB/s    in 1.0s    

2020-04-29 15:40:40 (12.7 MB/s) - ‘ngrok-stable-linux-amd64.zip.7’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [23]:
!pwd

/content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource/models/research


In [0]:
LOG_DIR = model_dir#+'Rerun'
get_ipython().system_raw(
    'tensorboard --logdir "{}" --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [25]:
LOG_DIR

'/content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource'

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

### Get Tensorboard link

In [80]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://918d511c.ngrok.io


## Train the model

Now all inputs are set up, just train the model.   This process may take a few hours.   Since we are saving the model training results (model.ckpt-* files) in our google drive (a persistent storage that will survice the restart of our colab VM instance), we can safely leave and return a few hours later. 

In [0]:
#################### SEND ALERT EMAIL AT FINISH WITH GMAIL OPTIONAL #####################
# To send email from Python from your google account, MUST 
# 1) Enable less secure app
# https://myaccount.google.com/lesssecureapps
# 2) Disable Unlock Capcha
# https://accounts.google.com/b/0/DisplayUnlockCaptcha

import smtplib

def SendEmail(msg):
    with open('/content/gdrive/My Drive/Colab Notebooks/pw.txt') as file:
        data = file.readlines()
        
    gmail_user = 'gehmaid@student.unimelb.edu.au'  
    gmail_password = data[0]


    sent_from = gmail_user  
    to = ['gehmaid@student.unimelb.edu.au']  
    subject = msg  
    body = '%s\n\n- Ghawady' % msg

    email_text = \
"""From: %s
To: %s
Subject: %s

%s
""" % (sent_from, ", ".join(to), subject, body)

    server = smtplib.SMTP("smtp.gmail.com", 587)
    server.ehlo()
    server.starttls()
    server.login(gmail_user, gmail_password)
    server.sendmail(sent_from, to, email_text)
    server.quit()

    print(f'Email: \n{email_text}')
    

In [94]:
num_steps = 5000
SendEmail("Colab train started")
!python '{model_dir}'/models/research/object_detection/model_main.py \
    --pipeline_config_path='{pipeline_fname}' \
    --model_dir='{trainedmodel_dir}' \
    --alsologtostderr \
    --num_train_steps='{num_steps}' \
    --num_eval_steps='{num_eval_steps}'
SendEmail("Colab train finished")

Email: 
From: gehmaid@student.unimelb.edu.au
To: gehmaid@student.unimelb.edu.au
Subject: Colab train started

Colab train started

- Ghawady

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W0429 16:14:15.219045 140127288534912 module_wrapper.py:139] From /content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource/models/research/object_detection/utils/config_util.py:102: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0429 16:14:15.223979 140127288534912 module_wrapper.py:139] From /content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource/models/research/object_detection/model_lib.py:628: The nam

In [30]:
!ls -ltra '{trainedmodel_dir}'

total 65921
-rw------- 1 root root 10669511 Apr 29 13:57 graph.pbtxt
-rw------- 1 root root    25149 Apr 29 13:57 model.ckpt-0.index
-rw------- 1 root root  7653760 Apr 29 13:57 model.ckpt-0.data-00000-of-00001
-rw------- 1 root root  5091964 Apr 29 13:57 model.ckpt-0.meta
-rw------- 1 root root    25149 Apr 29 14:07 model.ckpt-4122.index
-rw------- 1 root root  7653760 Apr 29 14:07 model.ckpt-4122.data-00000-of-00001
-rw------- 1 root root  5091964 Apr 29 14:07 model.ckpt-4122.meta
drwx------ 2 root root     4096 Apr 29 14:07 eval_0
-rw------- 1 root root  5091964 Apr 29 14:10 model.ckpt-5000.meta
-rw------- 1 root root    25149 Apr 29 14:10 model.ckpt-5000.index
-rw------- 1 root root  7653760 Apr 29 14:10 model.ckpt-5000.data-00000-of-00001
-rw------- 1 root root      176 Apr 29 14:10 checkpoint
drwx------ 3 root root     4096 Apr 29 14:10 export
-rw------- 1 root root 18508676 Apr 29 14:10 events.out.tfevents.1588168643.b4c3954872cb


# Section 5: Save and Convert Model Output

## Exporting a Trained Inference Graph
Once your training job is complete, you need to extract the newly trained inference graph, which will be later used to perform the object detection. This can be done as follows:

In [0]:
import os
import re
import numpy as np

output_directory = trainedmodel_dir + '/fine_tuned_model'
os.makedirs(output_directory, exist_ok=True)
!ls '{output_directory}'

In [96]:
lst = os.listdir(trainedmodel_dir)
# find the last model checkpoint file, i.e. model.ckpt-1000.meta
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')

last_model_path = os.path.join(trainedmodel_dir, last_model)
print(last_model_path)

/content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource/trainedmodel/ssd_mobilenet_v3_large_coco/2020_04_29/model.ckpt-15000


In [97]:
!echo creates the frozen inference graph in fine_tune_model
# there is an "Incomplete shape" message.  but we can safely ignore that. 

!python '{model_dir}'/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path='{pipeline_fname}' \
    --output_directory='{output_directory}' \
    --trained_checkpoint_prefix='{last_model_path}'

creates the frozen inference graph in fine_tune_model
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W0429 16:50:30.429763 140591431976832 module_wrapper.py:139] From /content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource/models/research/object_detection/export_inference_graph.py:145: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0429 16:50:30.438804 140591431976832 module_wrapper.py:139] From /content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource/models/research/object_detection/exporter.py:402: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W0429 

In [98]:
# https://medium.com/tensorflow/training-and-serving-a-realtime-mobile-object-detector-in-30-minutes-with-cloud-tpus-b78971cf1193
# create the tensorflow lite graph
!python '{model_dir}'/models/research/object_detection/export_tflite_ssd_graph.py \
    --pipeline_config_path='{pipeline_fname}' \
    --trained_checkpoint_prefix='{last_model_path}' \
    --output_directory='{output_directory}' \
    --add_postprocessing_op=true

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W0429 16:50:58.856555 140223803246464 module_wrapper.py:139] From /content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource/models/research/object_detection/export_tflite_ssd_graph.py:133: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0429 16:50:58.868901 140223803246464 module_wrapper.py:139] From /content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource/models/research/object_detection/export_tflite_ssd_graph_lib.py:193: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W0429 16:50:58.870098 140223803246464 mo

In [99]:
# https://medium.com/tensorflow/training-and-serving-a-realtime-mobile-object-detector-in-30-minutes-with-cloud-tpus-b78971cf1193
# create the tensorflow lite graph
!python '{model_dir}'/models/research/object_detection/export_tflite_ssd_graph.py \
    --pipeline_config_path='{pipeline_fname}' \
    --trained_checkpoint_prefix='{last_model_path}' \
    --output_directory='{output_directory}' \
    --add_postprocessing_op=true

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W0429 16:51:21.370703 139947338475392 module_wrapper.py:139] From /content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource/models/research/object_detection/export_tflite_ssd_graph.py:133: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0429 16:51:21.376410 139947338475392 module_wrapper.py:139] From /content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource/models/research/object_detection/export_tflite_ssd_graph_lib.py:193: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W0429 16:51:21.377127 139947338475392 mo

In [100]:
!echo "CONVERTING frozen graph to quantized TF Lite file..."
!tflite_convert \
  --output_file='{output_directory}/detect.tflite' \
  --graph_def_file='{output_directory}/tflite_graph.pb' \
  --inference_type=QUANTIZED_UINT8 \
  --input_arrays='normalized_input_image_tensor' \
  --output_arrays='TFLite_Detection_PostProcess,TFLite_Detection_PostProcess:1,TFLite_Detection_PostProcess:2,TFLite_Detection_PostProcess:3' \
  --mean_values=128 \
  --std_dev_values=128 \
  --input_shapes=1,300,300,3 \
  --change_concat_input_ranges=false \
  --allow_nudging_weights_to_use_fast_gemm_kernel=true \
  --allow_custom_ops
  #--output_arrays represent four arrays: detection_boxes, detection_classes, detection_scores, and num_detections.

CONVERTING frozen graph to quantized TF Lite file...
2020-04-29 16:51:56.573383: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-04-29 16:51:56.576123: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-04-29 16:51:56.576663: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Found device 0 with properties: 
name: Tesla P100-PCIE-16GB major: 6 minor: 0 memoryClockRate(GHz): 1.3285
pciBusID: 0000:00:04.0
2020-04-29 16:51:56.576914: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-04-29 16:51:56.578614: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2020-04-29 16:51:56.580526: I tensorflow/stream_executor/platform/default/dso_loa

In [101]:
'''
!echo "CONVERTING frozen graph to quantized TF Lite file..."
!tflite_convert \
  --output_file='{output_directory}/bus_labels_quantized.tflite' \
  --graph_def_file='{output_directory}/tflite_graph.pb' \
  --inference_type=QUANTIZED_UINT8 \
  --input_arrays='normalized_input_image_tensor' \
  --output_arrays='TFLite_Detection_PostProcess,TFLite_Detection_PostProcess:1,TFLite_Detection_PostProcess:2,TFLite_Detection_PostProcess:3' \
  --mean_values=128 \
  --std_dev_values=128 \
  --input_shapes=1,300,300,3 \
  --change_concat_input_ranges=false \
  --allow_nudging_weights_to_use_fast_gemm_kernel=true \
  --allow_custom_ops
'''

'\n!echo "CONVERTING frozen graph to quantized TF Lite file..."\n!tflite_convert   --output_file=\'{output_directory}/bus_labels_quantized.tflite\'   --graph_def_file=\'{output_directory}/tflite_graph.pb\'   --inference_type=QUANTIZED_UINT8   --input_arrays=\'normalized_input_image_tensor\'   --output_arrays=\'TFLite_Detection_PostProcess,TFLite_Detection_PostProcess:1,TFLite_Detection_PostProcess:2,TFLite_Detection_PostProcess:3\'   --mean_values=128   --std_dev_values=128   --input_shapes=1,300,300,3   --change_concat_input_ranges=false   --allow_nudging_weights_to_use_fast_gemm_kernel=true   --allow_custom_ops\n'

In [102]:
print(output_directory)
!ls -ltra '{output_directory}'
pb_fname = os.path.join(os.path.abspath(output_directory), "frozen_inference_graph.pb")  # this is tflite graph
!cp '{label_map_pbtxt_fname}' '{output_directory}'

/content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource/trainedmodel/ssd_mobilenet_v3_large_coco/2020_04_29/fine_tuned_model
total 53932
drwx------ 3 root root     4096 Apr 29 16:01 saved_model
-rw------- 1 root root     4593 Apr 29 16:01 pipeline.config
-rw------- 1 root root       77 Apr 29 16:02 label_map.pbtxt
-rw------- 1 root root  1575135 Apr 29 16:50 model.ckpt.meta
-rw------- 1 root root    17698 Apr 29 16:50 model.ckpt.index
-rw------- 1 root root  8956772 Apr 29 16:50 model.ckpt.data-00000-of-00001
-rw------- 1 root root       77 Apr 29 16:50 checkpoint
-rw------- 1 root root  9552216 Apr 29 16:50 frozen_inference_graph.pb
-rw------- 1 root root 25832584 Apr 29 16:51 tflite_graph.pbtxt
-rw------- 1 root root  9280380 Apr 29 16:51 tflite_graph.pb


# Section 6: Run inference test
Test with images in repository `object_detection/data/images/test` directory.

### Reinitialize required variables incase this notebook is not run from the beginning

In [103]:
%matplotlib inline
%tensorflow_version 1.x
!pip install numpy==1.17.4
!pip install --user gast==0.2.2
#!pip install imutils

In [104]:
'''
from google.colab import drive
drive.mount('/content/gdrive')
model_dir = '/content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource'

test_record_fname = model_dir + '/Data/annotations/test.record'
train_record_fname = model_dir + '/Data/annotations/train.record'
label_map_pbtxt_fname = model_dir + '/Data/annotations/label_map.pbtxt'
num_classes = 2

selected_model = 'ssd_mobilenet_v3_small_coco' #'ssd_mobilenet_v2_quantized'
curdate = '2020_04_29'
trainedmodel_dir = os.path.join(model_dir, 'trainedmodel', selected_model, curdate)
os.makedirs(trainedmodel_dir, exist_ok = True)

'''

"\nfrom google.colab import drive\ndrive.mount('/content/gdrive')\nmodel_dir = '/content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource'\n\ntest_record_fname = model_dir + '/Data/annotations/test.record'\ntrain_record_fname = model_dir + '/Data/annotations/train.record'\nlabel_map_pbtxt_fname = model_dir + '/Data/annotations/label_map.pbtxt'\nnum_classes = 2\n\nselected_model = 'ssd_mobilenet_v3_small_coco' #'ssd_mobilenet_v2_quantized'\ncurdate = '2020_04_29'\ntrainedmodel_dir = os.path.join(model_dir, 'trainedmodel', selected_model, curdate)\nos.makedirs(trainedmodel_dir, exist_ok = True)\n\n"

### Load model & test images for inference

In [105]:
import os
import glob

output_directory = trainedmodel_dir+'/fine_tuned_model'
pb_fname = os.path.join(os.path.abspath(output_directory), "frozen_inference_graph.pb")  # this is tflite graph

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = pb_fname
print(PATH_TO_CKPT)

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = label_map_pbtxt_fname

# If you want to test the code with your images, just add images files to the PATH_TO_TEST_IMAGES_DIR.
PATH_TO_TEST_IMAGES_DIR =  os.path.join(model_dir, "Data/val_cars")
print(PATH_TO_TEST_IMAGES_DIR)

assert os.path.isfile(pb_fname)
assert os.path.isfile(PATH_TO_LABELS)
TEST_IMAGE_PATHS = glob.glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, "*.jpg"))
assert len(TEST_IMAGE_PATHS) > 0, 'No image found in `{}`.'.format(PATH_TO_TEST_IMAGES_DIR)
print(TEST_IMAGE_PATHS)

/content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource/trainedmodel/ssd_mobilenet_v3_large_coco/2020_04_29/fine_tuned_model/frozen_inference_graph.pb
/content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource/Data/val_cars
['/content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource/Data/val_cars/1.jpg', '/content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource/Data/val_cars/2.jpg', '/content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource/Data/val_cars/3.jpg', '/content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource/Data/val_cars/4.jpg', '/content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource/Data/val_cars/5.jpg', '/content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource/Data/val_cars/6.jpg', '/content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource/Data/val_cars

### Inference

In [106]:
%cd '{model_dir}'/models/research/object_detection

import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops


# This is needed to display the images.
%matplotlib inline


from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util


detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')


label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(
    label_map, max_num_classes=num_classes, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)


def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {
                output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                'num_detections', 'detection_boxes', 'detection_scores',
                'detection_classes', 'detection_masks'
            ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                        tensor_name)
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(
                    tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(
                    tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(
                    tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [
                                           real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [
                                           real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image.shape[0], image.shape[1])
                detection_masks_reframed = tf.cast(
                    tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(
                    detection_masks_reframed, 0)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict,
                                   feed_dict={image_tensor: np.expand_dims(image, 0)})
            #print('output_dict',output_dict)
            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(
                output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict[
                'detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
            
    return output_dict

/content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource/models/research/object_detection


#### Draw bounding boxes on all test images

In [107]:
# running inferences.  This should show images with bounding boxes
%matplotlib inline

print('Running inferences on %s' % TEST_IMAGE_PATHS)

for image_path in TEST_IMAGE_PATHS:
    image = Image.open(image_path)
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np, detection_graph)
    #print('Returned output_dict',output_dict)
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks'),
        use_normalized_coordinates=True,
        line_thickness=2)
    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(image_np)


Output hidden; open in https://colab.research.google.com to view.

### Bounding box image crop

In [0]:
def get_bounding_box_coordinates(image,
                               ymin,
                               xmin,
                               ymax,
                               xmax,
                               use_normalized_coordinates=True):
  
  im_width, im_height = image.size
  if use_normalized_coordinates:
    (left, right, top, bottom) = (int(xmin * im_width), int(xmax * im_width),
                                  int(ymin * im_height), int(ymax * im_height))
  else:
    (left, right, top, bottom) = (xmin, xmax, ymin, ymax)

  return (left, top, right, bottom)  # tuple order matters for cropping


### Read Text from cropped image

#### Use pytesseract for Text Read (Bad)

config reference:

https://www.pyimagesearch.com/2018/09/17/opencv-ocr-and-text-recognition-with-tesseract/

In [109]:
!apt install tesseract-ocr
!apt install libtesseract-dev
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libtesseract-dev is already the newest version (4.00~git2288-10f4998a-2).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


In [0]:
import pytesseract
from pytesseract import Output
import cv2

import re

def read_image_text(img):
  print('image', type(img))
  img_np = load_image_into_numpy_array(img)
  crop = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)
  
  plt.figure(figsize=IMAGE_SIZE)
  plt.imshow(crop)
  crop = cv2.GaussianBlur(crop, (5, 5), 0)
  plt.figure(figsize=IMAGE_SIZE)
  plt.imshow(crop)
  #_, crop = cv2.threshold(crop, 100, 150, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
  #plt.figure(figsize=IMAGE_SIZE)
  #plt.imshow(crop)

  print('1=',pytesseract.image_to_string(crop, config='-l eng --oem 1 --psm 7'))
  print('2=',pytesseract.image_to_string(crop, config='--oem 3 --psm 3'))
  print('3=',pytesseract.image_to_string(crop))
  print('org 3=',pytesseract.image_to_data(crop, output_type=Output.DICT))
  print('org 1=',pytesseract.image_to_string(image_np, config='-l eng --oem 1 --psm 7'))
  print('org 2=',pytesseract.image_to_string(image_np, config='--oem 3 --psm 3'))
  print('org 3=',pytesseract.image_to_string(image_np))
  print('org 3=',pytesseract.image_to_data(image_np, output_type=Output.DICT))

  custom_config = r'-l eng --oem 1 --psm 7' #r'--oem 3 --psm 3'
  text = pytesseract.image_to_string(img, config=custom_config)
  print('text from image', text)
  return text


def read_image_text(img):
  (left, top, right, bottom) = (0,25,img.size[0],img.size[1])   
  # Grayscale, Gaussian blur, Otsu's threshold
  image = img #.crop((left, top, right, bottom))
  image = load_image_into_numpy_array(image)
  cv2.imwrite(os.path.join(model_dir,'orgcrop.jpg'),image)
  image = cv2.resize(image,(0,0),fx=7,fy=7)
  plt.imshow(image)
  cv2.imwrite(os.path.join(model_dir,'resized.jpg'),image)
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  cv2.imwrite(os.path.join(model_dir,'grayscale.jpg'),gray)
  plt.figure(figsize=IMAGE_SIZE)
  plt.imshow(gray)
  blur = cv2.GaussianBlur(gray, (3,3), 0)
  cv2.imwrite(os.path.join(model_dir,'GaussianBlur.jpg'),blur)
  plt.figure(figsize=IMAGE_SIZE)
  plt.imshow(blur)
  #thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
  #plt.figure(figsize=IMAGE_SIZE)
  #plt.imshow(thresh)

  # Morph open to remove noise and invert image
  kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
  opening = cv2.morphologyEx(blur, cv2.MORPH_OPEN, kernel, iterations=1)
  cv2.imwrite(os.path.join(model_dir,'morphologyEx.jpg'),opening)
  invert = 255 - opening
  cv2.imwrite(os.path.join(model_dir,'invert.jpg'),invert)
  plt.figure(figsize=IMAGE_SIZE)
  plt.imshow(invert)
  print('first tiral')
  #oem: OCR Engine modes => 1 - Neural nets LSTM engine only
  #psm: Page Segmentation Mode ==> 
  #     3    Fully automatic page segmentation, but no OSD. (Default)
  #     6    Assume a single uniform block of text.
  #     7    Treat the image as a single text line.
  print(pytesseract.image_to_string(invert, lang='eng', config='--psm 10 --oem 3 -c tessedit_char_whitelist=0123456789'))
  print('second trial',pytesseract.image_to_string(invert, config="-c tessedit_char_whitelist=0123456789 -oem 0"))
  print('try different read config')


	# in order to apply Tesseract v4 to OCR text we must supply
	# (1) a language, 
  # (2) an OEM flag of 1, indicating that the we
	# wish to use the LSTM neural net model for OCR,
	# (3) an OEM value, in this case, 7 which implies that we are
	# treating the ROI as a single line of text
  config = ("-l eng --oem 1 --psm 7")
  text = pytesseract.image_to_string(image, config=config)

	# strip out non-ASCII text so we can draw the text on the image
	# using OpenCV, then draw the text and a bounding box surrounding
	# the text region of the input image
  #text = "".join([c if ord(c) < 128 else "" for c in text]).strip()
  
  print('latest article::::',text)
  text = pytesseract.image_to_string(image, config=config)

  # Perform text extraction
  #for i in [1,3,4,5,6,7,8,9,10,11,12,13]:
  #  data = pytesseract.image_to_string(invert, lang='eng', config='--psm '+str(i))
  #  print('--psm '+str(i), data)
  
  print('antoher tuning')
  retval, image = cv2.threshold(image,200,255, cv2.THRESH_BINARY)
  cv2.imwrite(os.path.join(model_dir,'THRESH_BINARY.jpg'),image)
  plt.figure(figsize=IMAGE_SIZE)
  plt.imshow(image)
  image = cv2.GaussianBlur(image,(11,11),0)
  cv2.imwrite(os.path.join(model_dir,'GaussianBlur2.jpg'),image)
  plt.figure(figsize=IMAGE_SIZE)
  plt.imshow(image)
  image = cv2.medianBlur(image,9)
  cv2.imwrite(os.path.join(model_dir,'medianBlur.jpg'),image)
  plt.figure(figsize=IMAGE_SIZE)
  plt.imshow(image)
  data = pytesseract.image_to_string(image, lang='eng')
  print('last trial',pytesseract.image_to_string(image, config="-c tessedit_char_whitelist=0123456789 -oem 0"))
  

  return data


def read_image_text(img, plot=False):
  readings = []
  config = ("-l eng --oem 1 --psm 7")
  #(left, top, right, bottom) = (0,25,img.size[0],img.size[1])   
  # Grayscale, Gaussian blur, Otsu's threshold
  image = img #.crop((left, top, right, bottom))
  image = load_image_into_numpy_array(image)
  #cv2.imwrite(os.path.join(model_dir,'orgcrop.jpg'),image)
  #print('original image',pytesseract.image_to_string(image, config=config))
  readings.append(pytesseract.image_to_string(image, config=config))
  
  image = cv2.resize(image,(0,0),fx=7,fy=7)
  if plot:
    plt.imshow(image)
  #cv2.imwrite(os.path.join(model_dir,'resized.jpg'),image)
  #print('rescale image',pytesseract.image_to_string(image, config=config))
  readings.append(pytesseract.image_to_string(image, config=config))
  
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  #cv2.imwrite(os.path.join(model_dir,'grayscale.jpg'),gray)
  if plot:
    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(gray)
  #print('gray scale',pytesseract.image_to_string(gray, config=config))
  readings.append(pytesseract.image_to_string(gray, config=config))

  blur = cv2.GaussianBlur(gray, (3,3), 0)
  #cv2.imwrite(os.path.join(model_dir,'GaussianBlur.jpg'),blur)
  if plot:
    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(blur)
  #print('blur',pytesseract.image_to_string(blur, config=config))
  readings.append(pytesseract.image_to_string(blur, config=config))

  #thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
  #plt.figure(figsize=IMAGE_SIZE)
  #plt.imshow(thresh)

  # Morph open to remove noise and invert image
  kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
  opening = cv2.morphologyEx(blur, cv2.MORPH_OPEN, kernel, iterations=1)
  #cv2.imwrite(os.path.join(model_dir,'morphologyEx.jpg'),opening)
  #print('morphologyEx',pytesseract.image_to_string(kernel, config=config))
  readings.append(pytesseract.image_to_string(kernel, config=config))

  invert = 255 - opening
  #cv2.imwrite(os.path.join(model_dir,'invert.jpg'),invert)
  if plot:
    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(invert)
  #print('invert',pytesseract.image_to_string(invert, config=config))
  readings.append(pytesseract.image_to_string(invert, config=config))
  
  #print('another tuning')
  retval, image = cv2.threshold(image,200,255, cv2.THRESH_BINARY)
  #cv2.imwrite(os.path.join(model_dir,'THRESH_BINARY.jpg'),image)
  if plot:
    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(image)
  #print('THRESH_BINARY',pytesseract.image_to_string(image, config=config))
  readings.append(pytesseract.image_to_string(image, config=config))

  image = cv2.GaussianBlur(image,(11,11),0)
  #cv2.imwrite(os.path.join(model_dir,'GaussianBlur2.jpg'),image)
  if plot:
    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(image)
  #print('GaussianBlur2',pytesseract.image_to_string(image, config=config))
  readings.append(pytesseract.image_to_string(image, config=config))

  image = cv2.medianBlur(image,9)
  #cv2.imwrite(os.path.join(model_dir,'medianBlur.jpg'),image)
  if plot:
    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(image)
  #print('medianBlur',pytesseract.image_to_string(image, config=config))
  readings.append(pytesseract.image_to_string(image, config=config))

  data = pytesseract.image_to_string(image, lang='eng')
  #print('last trial',data)
  #print('last trial',pytesseract.image_to_string(image, config="-c tessedit_char_whitelist=0123456789 -oem 1"))
  readings.append(pytesseract.image_to_string(image, config="-c tessedit_char_whitelist=0123456789 -oem 1"))
  

  return readings

In [111]:
# running inferences.  This should show images with bounding boxes
%matplotlib inline

import pandas

consolidated_results = []
print('Running inferences on %s' % TEST_IMAGE_PATHS)

# Set the default threshold for accepting the bounding box
min_score_thresh=.5

for image_path in TEST_IMAGE_PATHS:
    print('processing image: ',image_path)
    image = Image.open(image_path)
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)

    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)

    # Actual detection
    output_dict = run_inference_for_single_image(image_np, detection_graph)
    
    # Read detection Results:
    # Predicted boxes for crop coordinates 
    # box coordinates (ymin, xmin, ymax, xmax) are relative to the image
    boxes = output_dict['detection_boxes']
    # Scores needed to filtered accepted objects based on threshold
    scores = output_dict['detection_scores']

    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks'),
        use_normalized_coordinates=True,
        line_thickness=2)
    
    #plt.figure(figsize=IMAGE_SIZE)
    #plt.imshow(image_np)

    # Reset image array for content reading
    image_np = load_image_into_numpy_array(image)

    for i in range(boxes.shape[0]):
        # Filter based on threshold
        if scores is None or scores[i] > min_score_thresh:
            # boxes[i] is the box which will be drawn
            ymin, xmin, ymax, xmax = tuple(boxes[i].tolist())
            class_name = category_index[output_dict['detection_classes'][i]]['name']

            bounding_box_coordinates = get_bounding_box_coordinates(image, ymin, xmin, ymax, xmax)

            # Crop image based on bounding box
            cropped_img = image.crop(bounding_box_coordinates)
            #plt.figure(figsize=IMAGE_SIZE)
            #plt.imshow(cropped_img)

            #read text from cropped image
            readings = read_image_text(cropped_img)
            print('row',[os.path.basename(image_path),class_name,scores[i]]+readings)
            consolidated_results.append([os.path.basename(image_path),class_name,scores[i]]+readings)

            #print ("This box is gonna get used", ymin, xmin, ymax, xmax ,boxes[i], output_dict['detection_classes'][i])
            #im_width, im_height = image.size

df = pandas.DataFrame(consolidated_results,columns=['image','object','Score','original','resized','grayscale','GaussianBlur','morphologyEx','invert','THRESH_BINARY','GaussianBlur2','medianBlur','numOnly'])


Running inferences on ['/content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource/Data/val_cars/1.jpg', '/content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource/Data/val_cars/2.jpg', '/content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource/Data/val_cars/3.jpg', '/content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource/Data/val_cars/4.jpg', '/content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource/Data/val_cars/5.jpg', '/content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource/Data/val_cars/6.jpg', '/content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource/Data/val_cars/7.jpg', '/content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource/Data/val_cars/8.jpg', '/content/gdrive/My Drive/UniMelb/Semester1_2020/Internship/SilverPond/FinalSource/Data/val_cars/9.jpg', '/content/gdrive/My Drive/UniMel

In [112]:
df

,image,object,Score,original,resized,grayscale,GaussianBlur,morphologyEx,invert,THRESH_BINARY,GaussianBlur2,medianBlur,numOnly
0,1.jpg,BusRun,0.600058,ae ia,aa,44,44,a,ses,44,a4,a4,
1,1.jpg,PlateNo,0.572444,"10V""4V)","| | vs , iA Vv v4 i; V | X",| 10 TMme v AV),"HOV"" 4V)",a,! 10 tans: a e3we,; | 0 Thee v AVX,"s10V""4V)","s10V""4V)",
2,2.jpg,BusRun,0.849831,35,35,35,35,a,35,35,,,
3,2.jpg,PlateNo,0.792659,,1 J \ | 4 \ | X,| | TORIA THE ” | a ie diialin,OV 4VX,a,Deir,| 10 / 4 ly,"1OV""4yyX",OV 4yX,
4,3.jpg,BusRun,0.920852,i!,hoe TE!,44,44,a,44,as,as,as,
5,3.jpg,PlateNo,0.736591,UX0%a45,,| m™ Q’ 44 5,IX ACE,a,un ACE,,,7 | eee r i,
6,4.jpg,BusRun,0.820921,(pene),,ae},ae},a,ae},i =30 :,,,
7,4.jpg,PlateNo,0.758515,lov*4yx,10 VY 4VX,""" 10 THE ""4V)",’ 10 THE vA4V),a,", 10 THE oD,",1OV* 4\ IX,"10V""4VX","10V""4VX",
8,5.jpg,BusRun,0.622727,ea.,,Bes,Bes,a,‘=. ay,,", =o",", =O",
9,5.jpg,PlateNo,0.555152,[XQ%4q,TXQ%4. +,TXQ%4. ‘,TXQ%4qi,a,POLE,,,,


####Use Google Vision API

https://cloud.google.com/vision/docs/ocr

In [113]:
!pip install --upgrade google-api-python-client

Requirement already up-to-date: google-api-python-client in /usr/local/lib/python3.6/dist-packages (1.8.2)


**Getting a Google API Credential**

Visit API console, choose "Credentials" on the left-hand menu. Choose "Create Credentials" and generate an API key for your application. Ideally restrict it by IP/domain, but for now, just left if blank.
Enter the key in this first executable cell:

In [0]:
import getpass
# API Key 
APIKEY = getpass.getpass()
#Need to 

In [0]:
# Import the base64 encoding library.
import base64
from io import BytesIO

# Get base64 encoding of image data
def encode_image(image):
  buffered = BytesIO()
  image.save(buffered, format="JPEG")
  img_str = base64.b64encode(buffered.getvalue()).decode('utf-8')
  return img_str

# Run Vision API
def gvision_detect_text(img):
  from googleapiclient.discovery import build
  vservice = build('vision', 'v1', developerKey=APIKEY)
  request = vservice.images().annotate(body={
          'requests': [{
                  'image': {
                      'content': encode_image(img)
                  },
                  'features': [{
                      'type': 'TEXT_DETECTION',
                      'maxResults': 3
                  }]
              }],
          })
  responses = request.execute(num_retries=3)
  try:
    return responses['responses'][0]['textAnnotations'][0]['description']
  except:
    print('Error occured while fetching results from API')
    return ''

In [0]:
# Process text read to fetch only the standard plate number without surrounding text
# https://en.wikipedia.org/wiki/Vehicle_registration_plates_of_Australia
import re

def process_plateno_text(text):
  # TODO: Need enhacement, for now only identify two sets of alphanumeric with space
  m = re.search('( ?[a-zA-Z0-9]){1,9} ( ?[a-zA-Z0-9]){1,9}', text)
  if m:
      return m.group()
  return ''

def process_busrun_text(text):
  # Filter only numbers
  m = re.search('( ?[0-9]){1,9}', text)
  if m:
      return m.group()
  return ''

print('plateno',process_plateno_text('ngscars\n10V 4VX\nVIC\nVICTORIA THE EDUCATION STATE\n'))
print('busrun',process_busrun_text('10V\n'))


In [0]:
# running inferences.  This should show images with bounding boxes
%matplotlib inline

import pandas

consolidated_results = []
print('Running inferences on %s' % TEST_IMAGE_PATHS)

# Set the default threshold for accepting the bounding box
min_score_thresh=.5

for image_path in TEST_IMAGE_PATHS:
    print('processing image: ',image_path)
    image = Image.open(image_path)
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)

    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)

    # Actual detection.
    output_dict = run_inference_for_single_image(image_np, detection_graph)
    
    # Read detection Results:
    # Predicted boxes for crop coordinates 
    # box coordinates (ymin, xmin, ymax, xmax) are relative to the image
    boxes = output_dict['detection_boxes']
    # Scores needed to filtered accepted objects based on threshold
    scores = output_dict['detection_scores']

    for i in range(boxes.shape[0]):
        # Filter based on threshold
        if scores is None or scores[i] > min_score_thresh:
            # boxes[i] is the box which will be drawn
            ymin, xmin, ymax, xmax = tuple(boxes[i].tolist())
            class_name = category_index[output_dict['detection_classes'][i]]['name']

            bounding_box_coordinates = get_bounding_box_coordinates(image, ymin, xmin, ymax, xmax)

            # Crop image based on bounding box
            cropped_img = image.crop(bounding_box_coordinates)
            #plt.figure(figsize=IMAGE_SIZE)
            #plt.imshow(cropped_img)

            #read text from cropped image
            readings = gvision_detect_text(cropped_img)
            #fine tune reading with appropriate regex
            if class_name == 'PlateNo':
              readings = process_plateno_text(readings)
            elif class_name == 'BusRun':
              readings = process_busrun_text(readings)

            if readings:
              #print('row',[os.path.basename(image_path),class_name,scores[i],readings])
              consolidated_results.append([os.path.basename(image_path),class_name,scores[i],readings])
            else:
              print('nothing returned',readings)

            #print ("This box is gonna get used", ymin, xmin, ymax, xmax ,boxes[i], output_dict['detection_classes'][i])
            #im_width, im_height = image.size

df2 = pandas.DataFrame(consolidated_results,columns=['image','object','Score','reading'])


In [0]:
df2

# Section 7: Further Enhacements:

- Try other object detection pretrained models from the available Zoo list
- Adjust Training Hyper parameters
- Train with more labeled images 
- Augment train images (different zooms, rotations, contrast, lighting) (Currently only default is used in the pipeline - horizontal flip & random crop is used)
- Explore other multi digit models like SVHN https://arxiv.org/pdf/1312.6082.pdf https://github.com/penny4860/SVHN-deep-digit-detector
- Build own multi digit model & train from public dataset (won't be as good as google vision)
- Text post processing improvement (to ensure regex covers all standard AUS plates)